<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/Hello_cu_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello cu_cat: Easy automatic feature engineering for messy dataframes on GPUs!
Daniel, Tanmoy, Alex, Leo


[  [LOOM](https://www.loom.com/share/10231b9dd07f4e979942fd10140e2dd0)  ] – press play and dive into analysis!

The latest [PyGraphistry[AI]](https://github.com/graphistry/pygraphistry/) release GPU accelerates its automatic feature encoding pipeline, by introducing a new member to the open source GPU dataframe ecosystem: cu_cat.

Cu_cat is our GPU-accelerated open source fork of the popular CPU Python  library dirty_cat.   Like dirty_cat, cu_cat makes it easy to convert messy dataframes filled with numbers, strings, and timestamps into numeric feature columns optimized for AI models. It adds interoperability for GPU dataframes and replaces key kernels and algorithms with faster and more scalable GPU variants. **Even on low-end GPUs, we are now able to tackle much larger datasets in the same amount of time – or for the first time! – with end-to-end pipelines. We typically save time with 3-5X speedups and will even see 10X+.**

In [ ]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11 cudf-cu11 cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.1/1.1 GB 123.7 MB/s eta 0:00:01

In [ ]:
import cuml,cudf
cuml.__version__

In [ ]:
!pip install -U --force git+https://github.com/graphistry/pygraphistry.git@cudf
!pip install -U git+https://github.com/graphistry/cu-cat.git@DT

In [ ]:
!git+http://github.com/graphistry/cu-cat/tarball/datetimeegg=package-0.03.0

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/
# !cp VERSION.txt /usr/local/lib/python3.9/dist-packages/cu_cat/

In [ ]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key

# graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


In [ ]:
!pip install dirty_cat

In [ ]:
gpu = nvidia-smi --query-gpu=gpu_name --format=csv
nvidia-smi -L

In [ ]:
X=1
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
from time import time


In [ ]:
# import cupy as cp
# del g
# cp._default_memory_pool.free_all_blocks()
winlogs.drop(columns='Time')

In [ ]:
# bench=pd.DataFrame(columns=['dataset','row','col','cc_time','dc_time'])
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet

bench=pd.DataFrame(columns=['dataset','row','col','cc_time','dc_time'])

winlogsA=pd.read_parquet('part.88.parquet')
n=[5000,10000,50000,100000,500000,1000000]
for p in np.arange(X):

    for ii,i in enumerate(n):

        winlogs=winlogsA.sample(i,replace=False).iloc[:,:6]
        winlogs=winlogs[['EventID','LogHost','Time']]
        # winlogs=winlogs.drop(columns='Time')
        g = graphistry.nodes(cudf.from_pandas(winlogs))
        t=time()
        g.featurize(feature_engine='cu_cat',memoize=False)
        t1=time()-t

        g = graphistry.nodes(winlogs)
        t=time()
        g.featurize(feature_engine='dirty_cat',memoize=False)
        t2=time()-t

        bench.loc[1+bench.shape[0]]=['winlogs',winlogs.shape[0],winlogs.shape[1],t1,t2]
        bench.to_csv('winlogs_'+str(gpu)+'_bench_'+timestr+'.txt',sep='\t',mode='a')


In [ ]:
from google.colab import files
files.download("bench.txt")